# Kaggle - WiDS Datathon 2022

## Modelo Nº1 - 

# 1. Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#model
from catboost import CatBoostRegressor

# 2. Data

In [2]:
#paths to the Github repo

train_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/train.csv'
test_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/test.csv'
submit_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/sample_solution.csv'

In [3]:
#train dataset

train = pd.read_csv(train_path)
train.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [4]:
#test dataset

test = pd.read_csv(test_path)
test.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,id
0,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75757
1,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75758
2,7,State_1,Commercial,Grocery_store_or_food_market,16138.0,1950.0,1.0,59.1,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75759
3,7,State_1,Commercial,Grocery_store_or_food_market,97422.0,1971.0,34.0,35.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75760
4,7,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,35.0,1.8,38,50.596774,...,0,29,5,2,0,340.0,330.0,22.8,126.0,75761


In [5]:
#sumbit

submit = pd.read_csv(submit_path)
submit.head()

,id,site_eui
0,75757,0.0
1,75758,0.0
2,75759,0.0
3,75760,0.0
4,75761,0.0


In [6]:
x_train = train.drop(['id','site_eui'], axis=1)
x_train.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [7]:
y_train = train['site_eui']

In [8]:
cat_columns = list(train.select_dtypes(include=['object']).columns)
cat_columns

['State_Factor', 'building_class', 'facility_type']

In [9]:
model_1_1 = CatBoostRegressor(cat_features = cat_columns)

In [10]:
model_1_1.fit(x_train,y_train)

Learning rate set to 0.08112
0:	learn: 57.0756939	total: 171ms	remaining: 2m 50s
1:	learn: 56.0578973	total: 201ms	remaining: 1m 40s
2:	learn: 55.1373297	total: 225ms	remaining: 1m 14s
3:	learn: 54.3617057	total: 253ms	remaining: 1m 3s
4:	learn: 53.4545968	total: 278ms	remaining: 55.3s
5:	learn: 52.8636022	total: 301ms	remaining: 49.8s
6:	learn: 52.1157152	total: 327ms	remaining: 46.4s
7:	learn: 51.5601133	total: 353ms	remaining: 43.8s
8:	learn: 51.1501897	total: 375ms	remaining: 41.2s
9:	learn: 50.6351067	total: 440ms	remaining: 43.6s
10:	learn: 50.1698617	total: 469ms	remaining: 42.2s
11:	learn: 49.8501390	total: 494ms	remaining: 40.7s
12:	learn: 49.5050299	total: 523ms	remaining: 39.7s
13:	learn: 49.2611639	total: 548ms	remaining: 38.6s
14:	learn: 49.0439836	total: 571ms	remaining: 37.5s
15:	learn: 48.7749204	total: 599ms	remaining: 36.9s
16:	learn: 48.5624329	total: 625ms	remaining: 36.2s
17:	learn: 48.3749491	total: 654ms	remaining: 35.7s
18:	learn: 48.1951292	total: 683ms	remaini

162:	learn: 43.1686538	total: 4.6s	remaining: 23.6s
163:	learn: 43.1544872	total: 4.63s	remaining: 23.6s
164:	learn: 43.1496060	total: 4.66s	remaining: 23.6s
165:	learn: 43.1420730	total: 4.69s	remaining: 23.6s
166:	learn: 43.1294966	total: 4.72s	remaining: 23.5s
167:	learn: 43.1123755	total: 4.74s	remaining: 23.5s
168:	learn: 43.0794897	total: 4.77s	remaining: 23.4s
169:	learn: 43.0581351	total: 4.79s	remaining: 23.4s
170:	learn: 43.0438265	total: 4.82s	remaining: 23.4s
171:	learn: 43.0383995	total: 4.85s	remaining: 23.3s
172:	learn: 43.0097721	total: 4.87s	remaining: 23.3s
173:	learn: 42.9979693	total: 4.9s	remaining: 23.3s
174:	learn: 42.9950604	total: 4.92s	remaining: 23.2s
175:	learn: 42.9858957	total: 4.95s	remaining: 23.2s
176:	learn: 42.9826989	total: 4.97s	remaining: 23.1s
177:	learn: 42.9800737	total: 4.99s	remaining: 23.1s
178:	learn: 42.9689704	total: 5.02s	remaining: 23s
179:	learn: 42.9514899	total: 5.05s	remaining: 23s
180:	learn: 42.9486752	total: 5.07s	remaining: 23s
1

321:	learn: 41.4534942	total: 9.09s	remaining: 19.1s
322:	learn: 41.4463212	total: 9.11s	remaining: 19.1s
323:	learn: 41.4385796	total: 9.14s	remaining: 19.1s
324:	learn: 41.4329015	total: 9.17s	remaining: 19s
325:	learn: 41.4317466	total: 9.19s	remaining: 19s
326:	learn: 41.4175290	total: 9.22s	remaining: 19s
327:	learn: 41.3975355	total: 9.24s	remaining: 18.9s
328:	learn: 41.3934858	total: 9.27s	remaining: 18.9s
329:	learn: 41.3907314	total: 9.3s	remaining: 18.9s
330:	learn: 41.3836092	total: 9.39s	remaining: 19s
331:	learn: 41.3723089	total: 9.43s	remaining: 19s
332:	learn: 41.3691066	total: 9.45s	remaining: 18.9s
333:	learn: 41.3677117	total: 9.48s	remaining: 18.9s
334:	learn: 41.3646813	total: 9.52s	remaining: 18.9s
335:	learn: 41.3599850	total: 9.55s	remaining: 18.9s
336:	learn: 41.3492365	total: 9.61s	remaining: 18.9s
337:	learn: 41.3320764	total: 9.64s	remaining: 18.9s
338:	learn: 41.3217949	total: 9.67s	remaining: 18.9s
339:	learn: 41.3148208	total: 9.7s	remaining: 18.8s
340:	

481:	learn: 40.5451864	total: 13.8s	remaining: 14.9s
482:	learn: 40.5364359	total: 13.8s	remaining: 14.8s
483:	learn: 40.5320953	total: 13.9s	remaining: 14.8s
484:	learn: 40.5280269	total: 13.9s	remaining: 14.8s
485:	learn: 40.5148110	total: 13.9s	remaining: 14.7s
486:	learn: 40.4992481	total: 14s	remaining: 14.7s
487:	learn: 40.4972000	total: 14s	remaining: 14.7s
488:	learn: 40.4937712	total: 14s	remaining: 14.7s
489:	learn: 40.4860460	total: 14.1s	remaining: 14.7s
490:	learn: 40.4818539	total: 14.1s	remaining: 14.6s
491:	learn: 40.4694104	total: 14.2s	remaining: 14.6s
492:	learn: 40.4537169	total: 14.2s	remaining: 14.6s
493:	learn: 40.4515424	total: 14.2s	remaining: 14.6s
494:	learn: 40.4496833	total: 14.2s	remaining: 14.5s
495:	learn: 40.4438361	total: 14.3s	remaining: 14.5s
496:	learn: 40.4437290	total: 14.3s	remaining: 14.5s
497:	learn: 40.4376991	total: 14.3s	remaining: 14.5s
498:	learn: 40.4376033	total: 14.4s	remaining: 14.4s
499:	learn: 40.4304844	total: 14.4s	remaining: 14.4s

643:	learn: 39.7521610	total: 18.8s	remaining: 10.4s
644:	learn: 39.7467982	total: 18.9s	remaining: 10.4s
645:	learn: 39.7459790	total: 18.9s	remaining: 10.3s
646:	learn: 39.7446469	total: 18.9s	remaining: 10.3s
647:	learn: 39.7430191	total: 18.9s	remaining: 10.3s
648:	learn: 39.7388223	total: 19s	remaining: 10.3s
649:	learn: 39.7385808	total: 19s	remaining: 10.2s
650:	learn: 39.7369073	total: 19s	remaining: 10.2s
651:	learn: 39.7360649	total: 19.1s	remaining: 10.2s
652:	learn: 39.7319745	total: 19.1s	remaining: 10.1s
653:	learn: 39.7286207	total: 19.1s	remaining: 10.1s
654:	learn: 39.7200320	total: 19.1s	remaining: 10.1s
655:	learn: 39.7188289	total: 19.2s	remaining: 10s
656:	learn: 39.7114808	total: 19.2s	remaining: 10s
657:	learn: 39.7113845	total: 19.2s	remaining: 9.99s
658:	learn: 39.7107998	total: 19.2s	remaining: 9.96s
659:	learn: 39.7077475	total: 19.3s	remaining: 9.93s
660:	learn: 39.7052398	total: 19.3s	remaining: 9.89s
661:	learn: 39.6948550	total: 19.3s	remaining: 9.86s
662

803:	learn: 39.1654371	total: 23.4s	remaining: 5.71s
804:	learn: 39.1607142	total: 23.5s	remaining: 5.68s
805:	learn: 39.1558449	total: 23.5s	remaining: 5.65s
806:	learn: 39.1506848	total: 23.5s	remaining: 5.62s
807:	learn: 39.1503250	total: 23.5s	remaining: 5.59s
808:	learn: 39.1475313	total: 23.6s	remaining: 5.56s
809:	learn: 39.1453826	total: 23.6s	remaining: 5.53s
810:	learn: 39.1317427	total: 23.6s	remaining: 5.5s
811:	learn: 39.1274729	total: 23.6s	remaining: 5.47s
812:	learn: 39.1265950	total: 23.7s	remaining: 5.44s
813:	learn: 39.1227057	total: 23.7s	remaining: 5.42s
814:	learn: 39.1221500	total: 23.7s	remaining: 5.38s
815:	learn: 39.1207110	total: 23.8s	remaining: 5.36s
816:	learn: 39.1167099	total: 23.8s	remaining: 5.33s
817:	learn: 39.1070692	total: 23.8s	remaining: 5.3s
818:	learn: 39.1042078	total: 23.8s	remaining: 5.27s
819:	learn: 39.1030158	total: 23.9s	remaining: 5.24s
820:	learn: 39.0969041	total: 23.9s	remaining: 5.21s
821:	learn: 39.0944021	total: 23.9s	remaining: 5

959:	learn: 38.6024152	total: 28s	remaining: 1.17s
960:	learn: 38.6000260	total: 28.1s	remaining: 1.14s
961:	learn: 38.5974074	total: 28.1s	remaining: 1.11s
962:	learn: 38.5937265	total: 28.1s	remaining: 1.08s
963:	learn: 38.5905095	total: 28.1s	remaining: 1.05s
964:	learn: 38.5862834	total: 28.2s	remaining: 1.02s
965:	learn: 38.5827566	total: 28.2s	remaining: 992ms
966:	learn: 38.5827054	total: 28.3s	remaining: 965ms
967:	learn: 38.5786184	total: 28.3s	remaining: 936ms
968:	learn: 38.5770810	total: 28.3s	remaining: 907ms
969:	learn: 38.5757052	total: 28.4s	remaining: 879ms
970:	learn: 38.5708172	total: 28.5s	remaining: 850ms
971:	learn: 38.5662165	total: 28.5s	remaining: 820ms
972:	learn: 38.5636810	total: 28.5s	remaining: 791ms
973:	learn: 38.5636708	total: 28.5s	remaining: 762ms
974:	learn: 38.5524619	total: 28.6s	remaining: 732ms
975:	learn: 38.5444459	total: 28.6s	remaining: 703ms
976:	learn: 38.5408699	total: 28.6s	remaining: 674ms
977:	learn: 38.5371260	total: 28.6s	remaining: 6

In [11]:
x_test = test.drop('id', axis=1)
x_test.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [12]:
submit['site_eui'] = model_1_1.predict(x_test)
submit.to_csv('modelo_1_1.csv',index=False)